In [1]:
from keras import backend as K
# K._get_available_gpus()

AttributeError: module 'keras.backend' has no attribute '_get_available_gpus'

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# import keras
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
import os, re, csv, math, codecs


MAX_NB_WORDS = 1000000

In [5]:
# path = 'drive/MyDrive/Colab Notebooks/data/cleaned.csv'
# ft_path = 'drive/MyDrive/Colab Notebooks/fasttext/cc.pl.300.vec'
ft_path = 'data/fasttext/cc.pl.300.vec'
path = 'data/limit_5K_per_type_order_by_id_desc/10_tt_split/cleaned.csv'


In [4]:
df_concat = pd.read_csv(path, sep=';')
df_concat['doc_len'] = df_concat['text'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(df_concat['doc_len'].mean() + df_concat['doc_len'].std()).astype(int)

raw_docs = df_concat['text'].tolist()
label_names = list(set(df_concat['type']))
num_classes = len(label_names)

print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(raw_docs)  #leaky
word_seq = tokenizer.texts_to_sequences(raw_docs)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq = sequence.pad_sequences(word_seq, maxlen=max_seq_len)

tokenizing input data...
dictionary size:  542531


# LOAD FASTTEXT EMBEDDINGS

In [6]:
#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open(ft_path, encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


2000001it [03:31, 9450.68it/s]

found 2000000 word vectors


In [7]:
embed_dim = 300 

#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 454368


In [8]:
print("sample words not found: ", np.random.choice(words_not_found, 10))

sample words not found:  ['rtł' 'pomnaizgawąęfaę' 'watpliwnśei' 'gbpnr' 'geąsoz' 'ośvirbdaam'
 'komentka' 'pożżęi' 'nalrżżsżżci' 'zarqu']


In [9]:
import random
from operator import itemgetter 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tensorflow as tf 
from time import time


def lable2vec(lable, label_names):
    res = np.zeros(len(label_names))
    res[label_names.index(lable)] = 1
    return res

  
def train_model(x_train, y_train, nb_words, embed_dim, params):
  #CNN architecture
  model = Sequential()
  model.add(Embedding(nb_words, embed_dim,
            weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
  model.add(Conv1D(params['num_filters'][0], params['cernel_size'], activation='relu', padding='same'))
  model.add(MaxPooling1D(2))
  model.add(Conv1D(params['num_filters'][1], params['cernel_size'], activation='relu', padding='same'))
  model.add(GlobalMaxPooling1D())
  model.add(Dropout(params['dropout']))
  model.add(Dense(params['dence_nnum'], activation='relu', kernel_regularizer=regularizers.l2(params['weight_decay'])))
  model.add(Dense(len(y_train[0]), activation='softmax'))  

  adam = tf.optimizers.Adam(params['lrate'])
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  # model.summary()

  # define callbacks
  early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
  callbacks_list = [early_stopping]
  hist = model.fit(x_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks_list, validation_split=params['validation_split'], shuffle=True, verbose=2)

  # hist = model.fit(x_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], validation_split=params['validation_split'], shuffle=True, verbose=2)
  
  return model, hist

In [10]:
train, test = train_test_split(df_concat, test_size=0.2, random_state=42)

x_train = itemgetter(train.index.tolist())(word_seq)
x_test = itemgetter(test.index.tolist())(word_seq)

y_train = np.array([lable2vec(lable, label_names) for lable in train['type'].tolist()])
y_test = np.array(test['type'].tolist())

In [11]:
# train_model(x_train, y_train, nb_words, embed_dim, 10, 0.0001, 128, 0.5, [300, 128], 9, 64, 0.0001)

In [12]:
# df_res = pd.DataFrame()
# df_res = pd.read_csv('drive/MyDrive/Colab Notebooks/data/w2v_cnn.csv', sep=';')
df_res = pd.read_csv('data/limit_5K_per_type_order_by_id_desc/20_results/w2v_cnn.csv', sep=';')

In [ ]:
epochs = 20
lrate = 0.001
batch_size = 64
dropout = 0.2
num_filters = [256, 128]
cernel_size = 3
dence_nnum = 64
weight_decay = 0.0001

validation_split = 0.2

# for dropout in [0.1, 0.3, 0.5, 0.7]:
# for num_filters in [[64, 64], [128, 64], [128, 128], [256, 128]]:
# for cernel_size in [3, 6, 9, 12]:
for dence_nnum in [32, 64, 128, 256]:
  print('dence_nnum:', dence_nnum)

  row = {
      'sample': 'cleaned',
      'epochs': epochs,
      'lrate': lrate,
      'batch_size': batch_size,
      'dropout': dropout,
      'num_filters': num_filters,
      'cernel_size': cernel_size,
      'dence_nnum': dence_nnum,
      'weight_decay': weight_decay,
      'validation_split': validation_split
  }

  start = time()
  model, hist = train_model(x_train, y_train, nb_words, embed_dim, row)

  row['ttime'] = (time() - start)/60
  row['val_loss'] = hist.history['val_loss'][-1]
  row['val_accuracy'] = hist.history['val_accuracy'][-1]

  df_res = df_res.append(row, ignore_index=True)
  df_res.to_csv('drive/MyDrive/Colab Notebooks/data/w2v_cnn.csv', index=False, sep=';')

In [ ]:
df_res